In [1]:
import pandas as pd  # For data manipulation and DataFrame handling
import numpy as np  # For numerical operations and handling missing values
from sklearn.model_selection import train_test_split  # For splitting data into training and testing sets
from sklearn.preprocessing import StandardScaler  # For feature scaling (optional for decision trees)
from sklearn.impute import SimpleImputer  # For handling missing values
import matplotlib.pyplot as plt  # For plotting
import seaborn as sns  # For more advanced plotting
from sklearn.tree import plot_tree  # For visualizing the decision tree
from sklearn.preprocessing import MinMaxScaler

import csv
import os

In [ ]:
#Opening the dataset
dataset = '../data/raw/ImmoEliza_dataset_no_nan.csv'
df = pd.read_csv(dataset)
df.isnull().sum()
df['municipality_code'] = df['municipality_code'].astype('int')
df['facades'] = df['facades'].astype('int')

print(df.head())

FileNotFoundError: [Errno 2] No such file or directory: '../data/raw/ImmoEliza_dataset_no_nan.csv'

In [ ]:
communes = '../data/raw/cities.csv'
df_communes = pd.read_csv(communes)
print(df_communes.columns)
df_communes.shape

Index(['name', 'zipCode', 'nisCode', 'province', 'main'], dtype='object')


(2721, 5)

In [1]:

# Create a dictionary from df_communes for faster lookup for both city localitys and provinces
communes_dict = dict(zip(df_communes['zipCode'], df_communes['name']))
provinces_dict = dict(zip(df_communes['zipCode'], df_communes['province']))  # Assuming column 'province' contains the province localitys

# Add new columns 'locality' and 'province' to df
df['locality'] = None  # Initialize the new column with None
df['province'] = None  # Initialize the province column with None


# Loop through each row in df
for index, row in df.iterrows():
    municipality_code = row['postal_code']  # Get the municipality code
    # Check if the municipality_code exists in the communes dictionary for city localitys
    if municipality_code in communes_dict:
        # Assign the corresponding entity locality to the 'locality' column
        df.at[index, 'locality'] = communes_dict[municipality_code]
        # Assign the corresponding province locality to the 'province' column
        df.at[index, 'province'] = provinces_dict[municipality_code]



# Display the updated DataFrame
print(df.head())


print(df.shape)
#df.to_csv('dataset_province_municipality_code.csv', index=False)  

NameError: name 'df_communes' is not defined

In [ ]:
import openpyxl
revenus = '../data/raw/revenus.xlsx'
dfrevenus = pd.read_excel(revenus, engine='openpyxl')

# Assuming dfrevenus is the DataFrame containing your data
dfrevenus['Average_Income_Per_Citizen'] = dfrevenus['MS_TOT_NET_INC'] / dfrevenus['MS_TOT_RESIDENTS']

# Filter for the year 2022
df_2022 = dfrevenus[dfrevenus['CD_YEAR'] == 2022]

# Group by 'CD_MUNTY_REFNIS' and calculate the average income per citizen for each municipality
income_municipality = df_2022.groupby('CD_MUNTY_REFNIS').agg({
    'MS_TOT_NET_INC': 'sum',  # Total net income for the municipality
    'MS_TOT_RESIDENTS': 'sum',  # Total number of residents in the municipality
}).reset_index()

# Calculate the average income per citizen for each municipality
income_municipality['Average_Income_Per_Citizen'] = income_municipality['MS_TOT_NET_INC'] / income_municipality['MS_TOT_RESIDENTS']

income_municipality.to_csv('income_municipality.csv', index=False) 

df = df.merge(
    income_municipality[['CD_MUNTY_REFNIS', 'Average_Income_Per_Citizen']],
    left_on='municipality_code',
    right_on='CD_MUNTY_REFNIS',
    how='left'
)


In [ ]:
# Get the value counts for the 'locality' column
name_counts = df['locality'].value_counts()
zip_counts = df['postal_code'].value_counts()
municipality_counts = df['municipality_code'].value_counts()
# Display the result
print(name_counts, zip_counts, municipality_counts)


locality
Westkapelle    785
Antwerpen      348
Zandvoorde     260
Gent           234
Brussel        173
              ... 
Evegnée          1
Neigem           1
Moresnet         1
Châtelineau      1
Roux             1
Name: count, Length: 840, dtype: int64 postal_code
8300    785
8400    260
9000    234
1000    173
1180    165
       ... 
1982      1
3054      1
1852      1
4560      1
6044      1
Name: count, Length: 852, dtype: int64 municipality_code
11002    913
31043    869
44021    424
35013    260
31005    214
        ... 
92045      1
84016      1
62026      1
85007      1
82036      1
Name: count, Length: 551, dtype: int64


In [ ]:

from sklearn.preprocessing import StandardScaler
import pandas as pd

# List of numerical columns (adjust as needed)
numerical_columns = ['bedrooms', 'price', 'livingarea', 'surfaceoftheplot', 'gardensurface', 'facades', 'Average_Income_Per_Citizen']

# Initialize StandardScaler
scaler = StandardScaler()

# Apply standardization
df[numerical_columns] = scaler.fit_transform(df[numerical_columns])


In [ ]:
'''
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

# List of numerical columns (adjust as needed)
numerical_columns = ['bedrooms', 'price', 'livingarea', 'surfaceoftheplot', 'gardensurface', 'facades', 'Average_Income_Per_Citizen']

# Initialize MinMaxScaler
scaler = MinMaxScaler()

# Apply normalization
df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

# Check the result
print(df.head())
'''

"\nfrom sklearn.preprocessing import MinMaxScaler\nimport pandas as pd\n\n# List of numerical columns (adjust as needed)\nnumerical_columns = ['bedrooms', 'price', 'livingarea', 'surfaceoftheplot', 'gardensurface', 'facades', 'Average_Income_Per_Citizen']\n\n# Initialize MinMaxScaler\nscaler = MinMaxScaler()\n\n# Apply normalization\ndf[numerical_columns] = scaler.fit_transform(df[numerical_columns])\n\n# Check the result\nprint(df.head())\n"

In [ ]:
df.columns

Index(['Unnamed: 0', 'bedrooms', 'postal_code', 'kitchen', 'facades', 'price',
       'furnished', 'terrace', 'fireplace', 'garden', 'gardensurface', 'pool',
       'livingarea', 'surfaceoftheplot', 'municipality_code',
       'apartment_block', 'ground_floor', 'country_cottage', 'mansion',
       'penthouse', 'exceptional_property', 'kot', 'loft', 'manor_house',
       'service_flat', 'chalet', 'as_new', 'good', 'just_renovated',
       'to_be_done_up', 'to_renovate', 'to_restore', 'price_sqm',
       'is_apartment', 'is_house', 'locality', 'province', 'CD_MUNTY_REFNIS',
       'Average_Income_Per_Citizen'],
      dtype='object')

In [ ]:
'''
# Filter out municipality codes
municipality_code_counts = df['municipality_code'].value_counts()
valid_municipality_codes = municipality_code_counts[municipality_code_counts >= 15].index
print(f"Original DataFrame shape: {df.shape}")
# Create a filtered DataFrame
df = df[df['municipality_code'].isin(valid_municipality_codes)]

# Print the result
print(f"Filtered DataFrame shape: {df.shape}")
'''

'\n# Filter out municipality codes\nmunicipality_code_counts = df[\'municipality_code\'].value_counts()\nvalid_municipality_codes = municipality_code_counts[municipality_code_counts >= 15].index\nprint(f"Original DataFrame shape: {df.shape}")\n# Create a filtered DataFrame\ndf = df[df[\'municipality_code\'].isin(valid_municipality_codes)]\n\n# Print the result\nprint(f"Filtered DataFrame shape: {df.shape}")\n'

In [ ]:

# Filter out postal codes
postal_code_counts = df['postal_code'].value_counts()
valid_postal_codes = postal_code_counts[postal_code_counts >= 20].index
print(f"Original DataFrame shape: {df.shape}")
# Create a filtered DataFrame
df = df[df['postal_code'].isin(valid_postal_codes)]

# Print the result
print(f"Original DataFrame shape: {df.shape}")


Original DataFrame shape: (12665, 39)
Original DataFrame shape: (8841, 39)


In [ ]:
'''
# Filter out names
name_counts = df['name'].value_counts()
valid_names = name_counts[name_counts >= 20].index
print(f"Original DataFrame shape: {df.shape}")
# Create a filtered DataFrame
df = df[df['name'].isin(valid_names)]

# Print the result
print(f"Original DataFrame shape: {df.shape}")
'''

'\n# Filter out names\nname_counts = df[\'name\'].value_counts()\nvalid_names = name_counts[name_counts >= 20].index\nprint(f"Original DataFrame shape: {df.shape}")\n# Create a filtered DataFrame\ndf = df[df[\'name\'].isin(valid_names)]\n\n# Print the result\nprint(f"Original DataFrame shape: {df.shape}")\n'

In [ ]:
df.columns

Index(['Unnamed: 0', 'bedrooms', 'postal_code', 'kitchen', 'facades', 'price',
       'furnished', 'terrace', 'fireplace', 'garden', 'gardensurface', 'pool',
       'livingarea', 'surfaceoftheplot', 'municipality_code',
       'apartment_block', 'ground_floor', 'country_cottage', 'mansion',
       'penthouse', 'exceptional_property', 'kot', 'loft', 'manor_house',
       'service_flat', 'chalet', 'as_new', 'good', 'just_renovated',
       'to_be_done_up', 'to_renovate', 'to_restore', 'price_sqm',
       'is_apartment', 'is_house', 'locality', 'province', 'CD_MUNTY_REFNIS',
       'Average_Income_Per_Citizen'],
      dtype='object')

In [ ]:
df.drop(['price_sqm'], axis=1, inplace=True)
df.drop(['Unnamed: 0'], axis=1, inplace=True)
df.drop(['kot'], axis=1, inplace=True)
df.drop(['loft'], axis=1, inplace=True)
df.drop(['apartment_block'], axis=1, inplace=True)
df.drop(['ground_floor'], axis=1, inplace=True)
df.drop(['country_cottage'], axis=1, inplace=True)
df.drop(['mansion'], axis=1, inplace=True)
df.drop(['penthouse'], axis=1, inplace=True)
df.drop(['exceptional_property'], axis=1, inplace=True)
df.drop(['manor_house'], axis=1, inplace=True)
df.drop(['service_flat'], axis=1, inplace=True)
df.drop(['chalet'], axis=1, inplace=True)
df.drop(['CD_MUNTY_REFNIS'], axis=1, inplace=True)

#df.drop(['surfaceoftheplot'], axis=1, inplace=True)


In [ ]:

#df = pd.get_dummies(df, columns=['municipality_code'], drop_first=False)
df = pd.get_dummies(df, columns=['postal_code'], drop_first=False)
#df = pd.get_dummies(df, columns=['name'], drop_first=False)
#df = pd.get_dummies(df, columns=['province'], drop_first=False)
#df.drop(['postal_code'], axis=1, inplace=True)
df.drop(['municipality_code'], axis=1, inplace=True)
df.drop(['locality'], axis=1, inplace=True)
df.drop(['province'], axis=1, inplace=True)

In [ ]:
'''# Calculate Q1 (25th percentile) and Q3 (75th percentile) for relevant columns
Q1 = df[['bedrooms', 'price', 'livingarea', 'surfaceoftheplot', 'gardensurface']].quantile(0.05)
Q3 = df[['bedrooms', 'price', 'livingarea', 'surfaceoftheplot', 'gardensurface']].quantile(0.95)
IQR = Q3 - Q1

# Define the outlier condition: values outside 1.5 * IQR
outliers_condition = ((df[['bedrooms', 'price', 'livingarea', 'surfaceoftheplot', 'gardensurface']] < (Q1 - 1.5 * IQR)) | 
                      (df[['bedrooms', 'price', 'livingarea', 'surfaceoftheplot', 'gardensurface']] > (Q3 + 1.5 * IQR)))

# Remove rows where any of the relevant columns contain outliers
df = df[~outliers_condition.any(axis=1)]
'''

"# Calculate Q1 (25th percentile) and Q3 (75th percentile) for relevant columns\nQ1 = df[['bedrooms', 'price', 'livingarea', 'surfaceoftheplot', 'gardensurface']].quantile(0.05)\nQ3 = df[['bedrooms', 'price', 'livingarea', 'surfaceoftheplot', 'gardensurface']].quantile(0.95)\nIQR = Q3 - Q1\n\n# Define the outlier condition: values outside 1.5 * IQR\noutliers_condition = ((df[['bedrooms', 'price', 'livingarea', 'surfaceoftheplot', 'gardensurface']] < (Q1 - 1.5 * IQR)) | \n                      (df[['bedrooms', 'price', 'livingarea', 'surfaceoftheplot', 'gardensurface']] > (Q3 + 1.5 * IQR)))\n\n# Remove rows where any of the relevant columns contain outliers\ndf = df[~outliers_condition.any(axis=1)]\n"

In [ ]:

from scipy.stats import zscore

# Calculate Z-scores for numeric columns
#z_scores = df[['bedrooms', 'price', 'livingarea', 'surfaceoftheplot', 'gardensurface', 'facades']].apply(zscore)
z_scores = df[['price', 'livingarea']].apply(zscore)
# Define a threshold for Z-scores (e.g., 3)
threshold = 3
print(f"Original DataFrame shape: {df.shape}")
# Filter the dataframe, keeping only the rows where all Z-scores are below the threshold
df = df[(z_scores < threshold).all(axis=1)]
print(f"After ZSCORE DataFrame shape: {df.shape}")

Original DataFrame shape: (8841, 179)
After ZSCORE DataFrame shape: (8592, 179)


In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop('price', axis=1)  # Features (all columns except 'price')
y = df['price']  # Target variable (price)

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Combine features and target back into DataFrames for X_train and X_test
df_train = X_train.copy()
df_train['price'] = y_train

df_test = X_test.copy()
df_test['price'] = y_test

# Check the shape of the splits
print(f"Training set size: {df_train.shape[0]} samples")
print(f"Test set size: {df_test.shape[0]} samples")


Training set size: 6873 samples
Test set size: 1719 samples


In [ ]:
'''from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardizing the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)  # Fit and transform on the training set
X_test = scaler.transform(X_test)       # Only transform the test set
'''

'from sklearn.model_selection import train_test_split\nfrom sklearn.preprocessing import StandardScaler\n\n# Splitting the data\nX_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)\n\n# Standardizing the data\nscaler = StandardScaler()\nX_train = scaler.fit_transform(X_train)  # Fit and transform on the training set\nX_test = scaler.transform(X_test)       # Only transform the test set\n'

In [ ]:
# Import necessary libraries
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Initialize the DecisionTreeRegressor model
model = DecisionTreeRegressor(random_state=42)

# Train the model using the training data
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the results
print(f"Mean Squared Error: {mse:.2f}")
print(f"R-squared: {r2:.2f}")


Mean Squared Error: 0.15
R-squared: 0.61


In [ ]:
# Get feature importances
feature_importances = model.feature_importances_

# Sort and display the features by importance
important_features = sorted(zip(X_train.columns, feature_importances), key=lambda x: x[1], reverse=True)
print("Feature importances:")
for feature, importance in important_features:
    print(f"{feature}: {importance:.4f}")


Feature importances:
livingarea: 0.4084
Average_Income_Per_Citizen: 0.2763
surfaceoftheplot: 0.0492
bedrooms: 0.0356
as_new: 0.0211
postal_code_1180: 0.0196
terrace: 0.0125
is_apartment: 0.0122
postal_code_8300: 0.0122
gardensurface: 0.0121
kitchen: 0.0113
facades: 0.0101
postal_code_1150: 0.0099
good: 0.0077
just_renovated: 0.0064
furnished: 0.0059
to_be_done_up: 0.0049
postal_code_9000: 0.0045
is_house: 0.0044
to_renovate: 0.0036
postal_code_8000: 0.0033
postal_code_1000: 0.0031
postal_code_1050: 0.0028
postal_code_8620: 0.0027
postal_code_1200: 0.0027
garden: 0.0027
postal_code_2000: 0.0027
postal_code_1160: 0.0026
postal_code_8380: 0.0024
fireplace: 0.0024
postal_code_3000: 0.0022
postal_code_1640: 0.0018
postal_code_2900: 0.0017
postal_code_8400: 0.0015
postal_code_9280: 0.0015
postal_code_8660: 0.0015
postal_code_8870: 0.0015
postal_code_2650: 0.0012
postal_code_8800: 0.0012
postal_code_1170: 0.0012
postal_code_9800: 0.0011
postal_code_2018: 0.0011
pool: 0.0011
postal_code_1040: 

In [ ]:
# Import necessary libraries
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Initialize the RandomForestRegressor model
model = RandomForestRegressor(random_state=42, n_estimators=100)  # You can adjust n_estimators as needed

# Train the model using the training data
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the results
print(f"Mean Squared Error: {mse:.2f}")
print(f"R-squared: {r2:.2f}")


Mean Squared Error: 0.10
R-squared: 0.74


In [ ]:
# Get feature importances
feature_importances = model.feature_importances_

# Print feature importances in descending order
for feature, importance in sorted(zip(X_train.columns, feature_importances), key=lambda x: x[1], reverse=True):
    print(f"{feature}: {importance:.4f}")


livingarea: 0.4129
Average_Income_Per_Citizen: 0.2764
surfaceoftheplot: 0.0483
bedrooms: 0.0338
as_new: 0.0191
postal_code_1180: 0.0172
is_apartment: 0.0146
facades: 0.0131
postal_code_8300: 0.0122
gardensurface: 0.0111
kitchen: 0.0105
terrace: 0.0105
good: 0.0086
postal_code_1150: 0.0082
just_renovated: 0.0058
furnished: 0.0057
postal_code_2000: 0.0051
postal_code_9000: 0.0048
to_renovate: 0.0042
to_be_done_up: 0.0040
postal_code_1050: 0.0038
is_house: 0.0029
pool: 0.0027
postal_code_1000: 0.0027
postal_code_8620: 0.0025
postal_code_1200: 0.0025
postal_code_8380: 0.0024
postal_code_8000: 0.0024
garden: 0.0023
postal_code_3000: 0.0022
postal_code_1950: 0.0021
postal_code_8400: 0.0020
fireplace: 0.0020
postal_code_8420: 0.0019
postal_code_1160: 0.0018
postal_code_8301: 0.0017
postal_code_8660: 0.0012
postal_code_2900: 0.0012
postal_code_1040: 0.0011
postal_code_2018: 0.0010
postal_code_2930: 0.0009
postal_code_1780: 0.0008
postal_code_8800: 0.0008
postal_code_8870: 0.0008
postal_code_19

In [ ]:
'''import matplotlib.pyplot as plt
import seaborn as sns

# Create a bar plot of feature importances
sns.barplot(x=feature_importances, y=X_train.columns)
plt.title("Feature Importances")
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.show()
'''

'import matplotlib.pyplot as plt\nimport seaborn as sns\n\n# Create a bar plot of feature importances\nsns.barplot(x=feature_importances, y=X_train.columns)\nplt.title("Feature Importances")\nplt.xlabel("Importance")\nplt.ylabel("Feature")\nplt.show()\n'

In [ ]:
'''# Import necessary libraries
from sklearn.tree import DecisionTreeRegressor, plot_tree
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score

# Initialize the DecisionTreeRegressor model
model = DecisionTreeRegressor(random_state=42)

# Train the model using the training data
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the results
print(f"Mean Squared Error: {mse:.2f}")
print(f"R-squared: {r2:.2f}")

# Visualize the decision tree
plt.figure(figsize=(200, 100))  # Adjust the size to your preference
plot_tree(model, filled=True, feature_names=X_train.columns, rounded=True)
plt.show()
'''

'# Import necessary libraries\nfrom sklearn.tree import DecisionTreeRegressor, plot_tree\nimport matplotlib.pyplot as plt\nfrom sklearn.metrics import mean_squared_error, r2_score\n\n# Initialize the DecisionTreeRegressor model\nmodel = DecisionTreeRegressor(random_state=42)\n\n# Train the model using the training data\nmodel.fit(X_train, y_train)\n\n# Make predictions on the test set\ny_pred = model.predict(X_test)\n\n# Evaluate the model\nmse = mean_squared_error(y_test, y_pred)\nr2 = r2_score(y_test, y_pred)\n\n# Print the results\nprint(f"Mean Squared Error: {mse:.2f}")\nprint(f"R-squared: {r2:.2f}")\n\n# Visualize the decision tree\nplt.figure(figsize=(200, 100))  # Adjust the size to your preference\nplot_tree(model, filled=True, feature_names=X_train.columns, rounded=True)\nplt.show()\n'